In [1]:
import pandas as pd
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import catboost
import lightgbm
import xgboost
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, Ridge
from sklearn.model_selection import ShuffleSplit, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train_top.csv', index_col='id')
test = pd.read_csv('test_top.csv', index_col='id')
y = train.skilled
train = train.drop(['skilled'], axis=1)
X = train
x_test = test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
X.shape

(99871, 618)

# LGBM

In [4]:
lgbm =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1)

In [5]:
%%time
lgbm.fit(X_train,y_train)

CPU times: user 14min 33s, sys: 11 s, total: 14min 44s
Wall time: 1min 56s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [6]:
accuracy_score(y_test, lgbm.predict(X_test))

0.830875659930821

<hr>

# XGB

In [18]:
xgb1 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=4, objective= 'binary:logistic', nthread=-1, scale_pos_weight=1, seed=27)
xgb2 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0, subsample=0.8, reg_alpha=9, nthread=6, seed=27)

<hr>

In [ ]:
%%time 
xgb1.fit(X_train,y_train)

In [12]:
accuracy_score(y_test, xgb1.predict(X_test))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8166454275138054

<hr>

In [20]:
%%time 
xgb2.fit(X_train,y_train)

CPU times: user 1h 49min 2s, sys: 14 s, total: 1h 49min 16s
Wall time: 18min 21s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=6, missing=None, n_estimators=1000,
       n_jobs=1, nthread=6, objective='binary:logistic', random_state=0,
       reg_alpha=9, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [21]:
accuracy_score(y_test, xgb2.predict(X_test))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8212876994963286

# CatBoost

In [241]:
cb = catboost.CatBoostClassifier(depth=6, n_estimators=5000) 

In [242]:
%%time
cb.fit(X_train, y_train)

Learning rate set to 0.013121
0:	learn: 0.6902322	total: 244ms	remaining: 20m 18s
1:	learn: 0.6873847	total: 495ms	remaining: 20m 36s
2:	learn: 0.6845648	total: 746ms	remaining: 20m 41s
3:	learn: 0.6818951	total: 1.04s	remaining: 21m 41s
4:	learn: 0.6792501	total: 1.3s	remaining: 21m 35s
5:	learn: 0.6767059	total: 1.57s	remaining: 21m 44s
6:	learn: 0.6741447	total: 1.81s	remaining: 21m 33s
7:	learn: 0.6716868	total: 2.06s	remaining: 21m 24s
8:	learn: 0.6692553	total: 2.33s	remaining: 21m 34s
9:	learn: 0.6669814	total: 2.61s	remaining: 21m 44s
10:	learn: 0.6647297	total: 2.87s	remaining: 21m 39s
11:	learn: 0.6624813	total: 3.16s	remaining: 21m 52s
12:	learn: 0.6603565	total: 3.43s	remaining: 21m 54s
13:	learn: 0.6583086	total: 3.69s	remaining: 21m 53s
14:	learn: 0.6562625	total: 3.96s	remaining: 21m 55s
15:	learn: 0.6542681	total: 4.23s	remaining: 21m 58s
16:	learn: 0.6523234	total: 4.49s	remaining: 21m 57s
17:	learn: 0.6502351	total: 4.74s	remaining: 21m 52s
18:	learn: 0.6483427	total:

155:	learn: 0.5441709	total: 43s	remaining: 22m 15s
156:	learn: 0.5438467	total: 43.3s	remaining: 22m 15s
157:	learn: 0.5435630	total: 43.6s	remaining: 22m 15s
158:	learn: 0.5432502	total: 43.9s	remaining: 22m 15s
159:	learn: 0.5429510	total: 44.1s	remaining: 22m 14s
160:	learn: 0.5426976	total: 44.4s	remaining: 22m 14s
161:	learn: 0.5423667	total: 44.7s	remaining: 22m 14s
162:	learn: 0.5420242	total: 44.9s	remaining: 22m 13s
163:	learn: 0.5417170	total: 45.2s	remaining: 22m 13s
164:	learn: 0.5414561	total: 45.5s	remaining: 22m 12s
165:	learn: 0.5411886	total: 45.8s	remaining: 22m 12s
166:	learn: 0.5408553	total: 46s	remaining: 22m 11s
167:	learn: 0.5405897	total: 46.3s	remaining: 22m 12s
168:	learn: 0.5403086	total: 46.6s	remaining: 22m 11s
169:	learn: 0.5400335	total: 46.9s	remaining: 22m 12s
170:	learn: 0.5397094	total: 47.2s	remaining: 22m 11s
171:	learn: 0.5394086	total: 47.4s	remaining: 22m 10s
172:	learn: 0.5391803	total: 47.7s	remaining: 22m 11s
173:	learn: 0.5389441	total: 48s

308:	learn: 0.5114949	total: 1m 27s	remaining: 22m 8s
309:	learn: 0.5113185	total: 1m 27s	remaining: 22m 7s
310:	learn: 0.5111678	total: 1m 28s	remaining: 22m 7s
311:	learn: 0.5110215	total: 1m 28s	remaining: 22m 7s
312:	learn: 0.5108529	total: 1m 28s	remaining: 22m 6s
313:	learn: 0.5106949	total: 1m 28s	remaining: 22m 5s
314:	learn: 0.5105528	total: 1m 29s	remaining: 22m 5s
315:	learn: 0.5103648	total: 1m 29s	remaining: 22m 5s
316:	learn: 0.5102134	total: 1m 29s	remaining: 22m 5s
317:	learn: 0.5100864	total: 1m 29s	remaining: 22m 4s
318:	learn: 0.5099488	total: 1m 30s	remaining: 22m 4s
319:	learn: 0.5098102	total: 1m 30s	remaining: 22m 5s
320:	learn: 0.5096794	total: 1m 30s	remaining: 22m 5s
321:	learn: 0.5095456	total: 1m 31s	remaining: 22m 5s
322:	learn: 0.5094257	total: 1m 31s	remaining: 22m 6s
323:	learn: 0.5092574	total: 1m 31s	remaining: 22m 5s
324:	learn: 0.5091200	total: 1m 32s	remaining: 22m 5s
325:	learn: 0.5089512	total: 1m 32s	remaining: 22m 4s
326:	learn: 0.5088199	total:

460:	learn: 0.4921541	total: 2m 12s	remaining: 21m 41s
461:	learn: 0.4920279	total: 2m 12s	remaining: 21m 41s
462:	learn: 0.4919229	total: 2m 12s	remaining: 21m 41s
463:	learn: 0.4918312	total: 2m 13s	remaining: 21m 41s
464:	learn: 0.4916871	total: 2m 13s	remaining: 21m 40s
465:	learn: 0.4915940	total: 2m 13s	remaining: 21m 40s
466:	learn: 0.4915046	total: 2m 13s	remaining: 21m 40s
467:	learn: 0.4914026	total: 2m 14s	remaining: 21m 40s
468:	learn: 0.4913034	total: 2m 14s	remaining: 21m 40s
469:	learn: 0.4912038	total: 2m 14s	remaining: 21m 39s
470:	learn: 0.4910893	total: 2m 15s	remaining: 21m 39s
471:	learn: 0.4909970	total: 2m 15s	remaining: 21m 39s
472:	learn: 0.4908549	total: 2m 15s	remaining: 21m 38s
473:	learn: 0.4907638	total: 2m 15s	remaining: 21m 38s
474:	learn: 0.4906467	total: 2m 16s	remaining: 21m 38s
475:	learn: 0.4905417	total: 2m 16s	remaining: 21m 38s
476:	learn: 0.4904505	total: 2m 16s	remaining: 21m 37s
477:	learn: 0.4903595	total: 2m 17s	remaining: 21m 37s
478:	learn

610:	learn: 0.4780754	total: 2m 56s	remaining: 21m 5s
611:	learn: 0.4780003	total: 2m 56s	remaining: 21m 5s
612:	learn: 0.4779046	total: 2m 56s	remaining: 21m 4s
613:	learn: 0.4778125	total: 2m 57s	remaining: 21m 4s
614:	learn: 0.4777163	total: 2m 57s	remaining: 21m 4s
615:	learn: 0.4776431	total: 2m 57s	remaining: 21m 3s
616:	learn: 0.4775793	total: 2m 57s	remaining: 21m 3s
617:	learn: 0.4774722	total: 2m 58s	remaining: 21m 3s
618:	learn: 0.4774083	total: 2m 58s	remaining: 21m 3s
619:	learn: 0.4773267	total: 2m 58s	remaining: 21m 3s
620:	learn: 0.4772534	total: 2m 59s	remaining: 21m 2s
621:	learn: 0.4771701	total: 2m 59s	remaining: 21m 2s
622:	learn: 0.4770794	total: 2m 59s	remaining: 21m 1s
623:	learn: 0.4770100	total: 2m 59s	remaining: 21m 1s
624:	learn: 0.4769379	total: 3m	remaining: 21m 1s
625:	learn: 0.4768443	total: 3m	remaining: 21m 1s
626:	learn: 0.4767550	total: 3m	remaining: 21m
627:	learn: 0.4766642	total: 3m 1s	remaining: 21m
628:	learn: 0.4766012	total: 3m 1s	remaining: 2

...

4300:	learn: 0.3635464	total: 21m 24s	remaining: 3m 28s
4301:	learn: 0.3635309	total: 21m 24s	remaining: 3m 28s
4302:	learn: 0.3634972	total: 21m 24s	remaining: 3m 28s
4303:	learn: 0.3634809	total: 21m 24s	remaining: 3m 27s
4304:	learn: 0.3634620	total: 21m 25s	remaining: 3m 27s
4305:	learn: 0.3634476	total: 21m 25s	remaining: 3m 27s
4306:	learn: 0.3634363	total: 21m 25s	remaining: 3m 26s
4307:	learn: 0.3634261	total: 21m 26s	remaining: 3m 26s
4308:	learn: 0.3634106	total: 21m 26s	remaining: 3m 26s
4309:	learn: 0.3633855	total: 21m 26s	remaining: 3m 26s
4310:	learn: 0.3633721	total: 21m 27s	remaining: 3m 25s
4311:	learn: 0.3633567	total: 21m 27s	remaining: 3m 25s
4312:	learn: 0.3633326	total: 21m 27s	remaining: 3m 25s
4313:	learn: 0.3633021	total: 21m 28s	remaining: 3m 24s
4314:	learn: 0.3632884	total: 21m 28s	remaining: 3m 24s
4315:	learn: 0.3632778	total: 21m 28s	remaining: 3m 24s
4316:	learn: 0.3632638	total: 21m 29s	remaining: 3m 23s
4317:	learn: 0.3632440	total: 21m 29s	remaining:

4448:	learn: 0.3610744	total: 22m 9s	remaining: 2m 44s
4449:	learn: 0.3610624	total: 22m 9s	remaining: 2m 44s
4450:	learn: 0.3610349	total: 22m 9s	remaining: 2m 44s
4451:	learn: 0.3610150	total: 22m 10s	remaining: 2m 43s
4452:	learn: 0.3609955	total: 22m 10s	remaining: 2m 43s
4453:	learn: 0.3609739	total: 22m 10s	remaining: 2m 43s
4454:	learn: 0.3609568	total: 22m 10s	remaining: 2m 42s
4455:	learn: 0.3609376	total: 22m 11s	remaining: 2m 42s
4456:	learn: 0.3609192	total: 22m 11s	remaining: 2m 42s
4457:	learn: 0.3609033	total: 22m 11s	remaining: 2m 41s
4458:	learn: 0.3608935	total: 22m 12s	remaining: 2m 41s
4459:	learn: 0.3608757	total: 22m 12s	remaining: 2m 41s
4460:	learn: 0.3608592	total: 22m 12s	remaining: 2m 41s
4461:	learn: 0.3608489	total: 22m 13s	remaining: 2m 40s
4462:	learn: 0.3608317	total: 22m 13s	remaining: 2m 40s
4463:	learn: 0.3608143	total: 22m 13s	remaining: 2m 40s
4464:	learn: 0.3607982	total: 22m 14s	remaining: 2m 39s
4465:	learn: 0.3607887	total: 22m 14s	remaining: 2m

4595:	learn: 0.3586874	total: 22m 53s	remaining: 2m
4596:	learn: 0.3586690	total: 22m 54s	remaining: 2m
4597:	learn: 0.3586453	total: 22m 54s	remaining: 2m
4598:	learn: 0.3586276	total: 22m 54s	remaining: 1m 59s
4599:	learn: 0.3586100	total: 22m 55s	remaining: 1m 59s
4600:	learn: 0.3585954	total: 22m 55s	remaining: 1m 59s
4601:	learn: 0.3585768	total: 22m 55s	remaining: 1m 58s
4602:	learn: 0.3585601	total: 22m 56s	remaining: 1m 58s
4603:	learn: 0.3585440	total: 22m 56s	remaining: 1m 58s
4604:	learn: 0.3585160	total: 22m 56s	remaining: 1m 58s
4605:	learn: 0.3585006	total: 22m 57s	remaining: 1m 57s
4606:	learn: 0.3584803	total: 22m 57s	remaining: 1m 57s
4607:	learn: 0.3584685	total: 22m 57s	remaining: 1m 57s
4608:	learn: 0.3584460	total: 22m 57s	remaining: 1m 56s
4609:	learn: 0.3584332	total: 22m 58s	remaining: 1m 56s
4610:	learn: 0.3584256	total: 22m 58s	remaining: 1m 56s
4611:	learn: 0.3584028	total: 22m 58s	remaining: 1m 55s
4612:	learn: 0.3583899	total: 22m 59s	remaining: 1m 55s
4613

4743:	learn: 0.3563540	total: 23m 39s	remaining: 1m 16s
4744:	learn: 0.3563436	total: 23m 39s	remaining: 1m 16s
4745:	learn: 0.3563233	total: 23m 39s	remaining: 1m 15s
4746:	learn: 0.3563150	total: 23m 40s	remaining: 1m 15s
4747:	learn: 0.3562931	total: 23m 40s	remaining: 1m 15s
4748:	learn: 0.3562804	total: 23m 40s	remaining: 1m 15s
4749:	learn: 0.3562656	total: 23m 41s	remaining: 1m 14s
4750:	learn: 0.3562546	total: 23m 41s	remaining: 1m 14s
4751:	learn: 0.3562376	total: 23m 41s	remaining: 1m 14s
4752:	learn: 0.3562246	total: 23m 41s	remaining: 1m 13s
4753:	learn: 0.3562170	total: 23m 42s	remaining: 1m 13s
4754:	learn: 0.3562045	total: 23m 42s	remaining: 1m 13s
4755:	learn: 0.3561884	total: 23m 42s	remaining: 1m 12s
4756:	learn: 0.3561727	total: 23m 43s	remaining: 1m 12s
4757:	learn: 0.3561538	total: 23m 43s	remaining: 1m 12s
4758:	learn: 0.3561381	total: 23m 43s	remaining: 1m 12s
4759:	learn: 0.3561164	total: 23m 44s	remaining: 1m 11s
4760:	learn: 0.3560931	total: 23m 44s	remaining:

4893:	learn: 0.3540641	total: 24m 24s	remaining: 31.7s
4894:	learn: 0.3540493	total: 24m 25s	remaining: 31.4s
4895:	learn: 0.3540359	total: 24m 25s	remaining: 31.1s
4896:	learn: 0.3540294	total: 24m 25s	remaining: 30.8s
4897:	learn: 0.3540177	total: 24m 26s	remaining: 30.5s
4898:	learn: 0.3540056	total: 24m 26s	remaining: 30.2s
4899:	learn: 0.3539871	total: 24m 26s	remaining: 29.9s
4900:	learn: 0.3539783	total: 24m 27s	remaining: 29.6s
4901:	learn: 0.3539662	total: 24m 27s	remaining: 29.3s
4902:	learn: 0.3539499	total: 24m 27s	remaining: 29s
4903:	learn: 0.3539376	total: 24m 28s	remaining: 28.7s
4904:	learn: 0.3539251	total: 24m 28s	remaining: 28.4s
4905:	learn: 0.3539130	total: 24m 28s	remaining: 28.1s
4906:	learn: 0.3539000	total: 24m 28s	remaining: 27.8s
4907:	learn: 0.3538850	total: 24m 29s	remaining: 27.5s
4908:	learn: 0.3538672	total: 24m 29s	remaining: 27.2s
4909:	learn: 0.3538513	total: 24m 29s	remaining: 26.9s
4910:	learn: 0.3538285	total: 24m 30s	remaining: 26.6s
4911:	learn:

In [57]:
accuracy_score(y_test, cb.predict(X_test))

0.8111839310637782

# RidgeReg

In [17]:
rg0 = Ridge(alpha=0.0001)

In [18]:
%%time
rg0.fit(X_train, y_train)

CPU times: user 60 s, sys: 10.8 s, total: 1min 10s
Wall time: 24.9 s


Ridge(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [23]:
%%time
accuracy_score(y_test, (rg0.predict(X_test.fillna(0)) > 0.5).astype('int'))

CPU times: user 400 ms, sys: 150 ms, total: 550 ms
Wall time: 562 ms


0.8098792402451606

# LinearRegression

In [11]:
from sklearn.linear_model import LinearRegression

In [60]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [61]:
%%time
accuracy_score(y_test, (reg.predict(X_test) > 0.5).astype('int'))

CPU times: user 2.5 s, sys: 1.28 s, total: 3.78 s
Wall time: 3.51 s


0.7989865889920504

# Let's go to the TOP 1

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
train, valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

__models__

In [12]:
lgbm1 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1)
lgbm2 =  lightgbm.LGBMClassifier(n_estimators=1000, learning_rate=0.1, max_depth=6, reg_alpha=10, min_child_weight=5, n_jobs=-1, random_state=1)
xgb1 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=4, objective= 'binary:logistic', nthread=-1, scale_pos_weight=1, seed=27)
xgb2 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0, subsample=0.8, reg_alpha=9, nthread=6, seed=27)
cb = catboost.CatBoostClassifier(depth=8, n_estimators=1500, verbose=False) 
rg = Ridge(alpha=0.0001)
reg = LinearRegression()

In [13]:
for clf in tqdm_notebook([lgbm1, lgbm2, xgb1, xgb2, cb, rg, reg]):
    clf.fit(train, y_train)

Learning rate set to 0.031525



__meta features for train meta and predict__

In [14]:
preds1 = lgbm1.predict(valid)
preds2 = lgbm2.predict(valid)
preds3 = xgb1.predict(valid)
preds4 = xgb2.predict(valid)
preds5 = cb.predict(valid)
preds6 = rg.predict(valid)
preds7 = reg.predict(valid)

In [17]:
test_preds1 = lgbm1.predict(X_test)
test_preds2 = lgbm2.predict(X_test)
test_preds3 = xgb1.predict(X_test)
test_preds4 = xgb2.predict(X_test)
test_preds5 = cb.predict(X_test)
test_preds6 = rg.predict(X_test.fillna(0))
test_preds7 = reg.predict(X_test.fillna(0))

In [18]:
st_preds = np.column_stack((preds1, preds2, preds3, preds4, preds5, preds6, preds7))
st_preds_test = np.column_stack((test_preds1, test_preds2, test_preds3, test_preds4, test_preds5, test_preds6, test_preds7))

__unite meta with features__

In [19]:
df_meta_val = pd.DataFrame(st_preds, index=valid.index)
df_meta_test = pd.DataFrame(st_preds_test, index=X_test.index)

In [20]:
meta_valid = pd.concat([valid, df_meta_val], axis=1, join_axes=[valid.index])

In [21]:
meta_X_test = pd.concat([X_test, df_meta_test], axis=1, join_axes=[X_test.index])

__meta model__

In [41]:
meta_model = cb
meta_model.fit(meta_valid, y_valid)

Learning rate set to 0.031525


In [42]:
ans = meta_model.predict(meta_X_test.fillna(0))

In [43]:
accuracy_score(y_test, (ans > 0.5).astype('int') )

0.8327871836883306